# Mesh generation for tutorial 07

In [ ]:
import dolfinx.io
import dolfinx.mesh
import gmsh
import mpi4py
import numpy as np

In [ ]:
import multiphenicsx.io
import multiphenicsx.mesh

### Geometrical parameters

In [ ]:
r = 1
lcar = 2

### Create mesh

Generate a simple mesh consisting in an hexagon discretized with six equilateral triangle cells.

In [ ]:
gmsh.initialize()
gmsh.model.add("mesh")
points = [
    gmsh.model.geo.addPoint(np.cos(t / 3 * np.pi), np.sin(t / 3 * np.pi), 0.0, lcar) for t in range(6)]
lines = [gmsh.model.geo.addLine(points[t], points[(t + 1) % 6]) for t in range(6)]
line_loop = gmsh.model.geo.addCurveLoop(lines)
domain = gmsh.model.geo.addPlaneSurface([line_loop])
gmsh.model.geo.synchronize()
gmsh.model.addPhysicalGroup(1, lines, 0)
gmsh.model.addPhysicalGroup(2, [domain], 0)
gmsh.model.mesh.generate(2)

In [ ]:
mesh, subdomains, boundaries = multiphenicsx.mesh.gmsh_to_fenicsx(gmsh.model, gdim=2)
gmsh.finalize()

In [ ]:
multiphenicsx.io.plot_mesh(mesh)

### Create cell restrictions

Define mesh tags on cells, which are equal to one on all cells.

In [ ]:
cell_entities_all = dolfinx.mesh.locate_entities(
    mesh, mesh.topology.dim, lambda x: np.full((x.shape[1], ), True))
cell_values_all = np.full(cell_entities_all.shape, 1, dtype=np.intc)
cell_restriction_all = dolfinx.mesh.MeshTags(
    mesh, mesh.topology.dim, cell_entities_all, cell_values_all)
cell_restriction_all.name = "cell_restriction_all"

In [ ]:
multiphenicsx.io.plot_mesh_tags(cell_restriction_all)

Define mesh tags on cells, which are equal to one on one half of the cells

In [ ]:
eps = np.finfo(float).eps
cell_entities_subset = dolfinx.mesh.locate_entities(
    mesh, mesh.topology.dim,
    lambda x: np.logical_or(x[0] < eps, np.logical_and(x[1] < eps, x[0] < 0.5 + eps)))
cell_values_subset = np.full(cell_entities_subset.shape, 1, dtype=np.intc)
cell_restriction_subset = dolfinx.mesh.MeshTags(
    mesh, mesh.topology.dim, cell_entities_subset, cell_values_subset)
cell_restriction_subset.name = "cell_restriction_subset"

In [ ]:
multiphenicsx.io.plot_mesh_tags(cell_restriction_subset)

### Create facet restrictions

Define mesh tags on facets, which are equal to one on all facets

In [ ]:
facet_entities_all = dolfinx.mesh.locate_entities(
    mesh, mesh.topology.dim - 1, lambda x: np.full((x.shape[1], ), True))
facet_values_all = np.full(facet_entities_all.shape, 1, dtype=np.intc)
facet_restriction_all = dolfinx.mesh.MeshTags(
    mesh, mesh.topology.dim - 1, facet_entities_all, facet_values_all)
facet_restriction_all.name = "facet_restriction_all"

In [ ]:
multiphenicsx.io.plot_mesh_tags(facet_restriction_all)

Define mesh tags on facets, which are equal to one on two facets

In [ ]:
facet_entities_subset = dolfinx.mesh.locate_entities(
    mesh, mesh.topology.dim - 1, lambda x: np.fabs(x[1] + np.sqrt(3) * x[0]) < 0.01)
facet_values_subset = np.full(facet_entities_subset.shape, 1, dtype=np.intc)
facet_restriction_subset = dolfinx.mesh.MeshTags(
    mesh, mesh.topology.dim - 1, facet_entities_subset, facet_values_subset)
facet_restriction_subset.name = "facet_restriction_subset"

In [ ]:
multiphenicsx.io.plot_mesh_tags(facet_restriction_subset)

### Save

In [ ]:
with dolfinx.io.XDMFFile(mpi4py.MPI.COMM_WORLD, "hexagon.xdmf", "w") as output:
    output.write_mesh(mesh)
    output.write_meshtags(cell_restriction_all)
    output.write_meshtags(cell_restriction_subset)
    mesh.topology.create_connectivity(mesh.topology.dim - 1, mesh.topology.dim)
    output.write_meshtags(facet_restriction_all)
    output.write_meshtags(facet_restriction_subset)